# Spylon Test

In [1]:
%%init_spark
launcher.jars = ["file:///opt/spark-data/libs/xgb_1_12/xgboost4j_2.12-1.0.0.jar", "file:///opt/spark-data/libs/xgb_1+12/xgboost4j-spark_2.12-1.0.0.jar"]
launcher.master = "spark://spark-master:7077"
launcher.conf.spark.app.name = "Test App"
launcher.conf.spark.executor.cores = 2
launcher.conf.spark.executor.memory = "2g"

In [ ]:
%%init_spark
launcher.packages = ["ml.dmlc.xgboost4j_2.12:1.0.0"]
launcher.packages = ["ml.dmlc.xgboost4j-spark_2.12:1.0.0"]

In [ ]:
spark

In [2]:
// the import starts a spark sesh on local
// Need to change so taht we can create the session later
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{DoubleType, StringType, StructField, StructType}

Intitializing Scala interpreter ...

Spark Web UI available at http://c046525e0569:4040
SparkContext available as 'sc' (version = 3.0.0, master = spark://spark-master:7077, app id = app-20200620073818-0003)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{DoubleType, StringType, StructField, StructType}


In [ ]:
%ls /opt/spark-data/

In [3]:
val spark = SparkSession.builder().config("spark.executor.memory", "2g")
                    .config("spark.driver.memory", "4g")
                    .config("spark.jars.packages","ml.dmlc.xgboost4j_2.11:1.0.0")
                    .config("spark.jars.packages","ml.dmlc.xgboost4j-spark_2.11:1.0.0")
                    .master("spark://spark-master:7077").getOrCreate();

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@34deabbf


In [ ]:
//:require file:/opt/spark-data/libs/xgb_1_12/xgboost4j_2.12-1.0.0.jar
//:require file:/opt/spark-data/libs/xgb_1+12/xgboost4j-spark_2.12-1.0.0.jar

In [4]:
val schema = new StructType(Array(
  StructField("sepal length", DoubleType, true),
  StructField("sepal width", DoubleType, true),
  StructField("petal length", DoubleType, true),
  StructField("petal width", DoubleType, true),
  StructField("class", StringType, true)))

schema: org.apache.spark.sql.types.StructType = StructType(StructField(sepal length,DoubleType,true), StructField(sepal width,DoubleType,true), StructField(petal length,DoubleType,true), StructField(petal width,DoubleType,true), StructField(class,StringType,true))


In [5]:
val rawInput = spark.read.schema(schema).csv("/opt/spark-data/train.csv")

rawInput: org.apache.spark.sql.DataFrame = [sepal length: double, sepal width: double ... 3 more fields]


In [6]:
import org.apache.spark.ml.feature.StringIndexer
val stringIndexer = new StringIndexer().
  setInputCol("class").
  setOutputCol("classIndex").
  fit(rawInput)
val labelTransformed = stringIndexer.transform(rawInput).drop("class")

import org.apache.spark.ml.feature.StringIndexer
stringIndexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_c3a1fa5216d6, handleInvalid=error
labelTransformed: org.apache.spark.sql.DataFrame = [sepal length: double, sepal width: double ... 3 more fields]


In [7]:
import org.apache.spark.ml.feature.VectorAssembler
val vectorAssembler = new VectorAssembler().
  setInputCols(Array("sepal length", "sepal width", "petal length", "petal width")).
  setOutputCol("features")
val xgbInput = vectorAssembler.transform(labelTransformed).select("features", "classIndex")

import org.apache.spark.ml.feature.VectorAssembler
vectorAssembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_d8c04fc2b16c, handleInvalid=error, numInputCols=4
xgbInput: org.apache.spark.sql.DataFrame = [features: vector, classIndex: double]


In [9]:
import ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier

<console>: 28: error: object spark is not a member of package ml.dmlc.xgboost4j.scala

In [8]:
val xgbParam = Map("eta" -> 0.1f,
      "missing" -> -999,
      "objective" -> "multi:softprob",
      "num_class" -> 3,
      "num_round" -> 100,
      "num_workers" -> 2)
val xgbClassifier = new XGBoostClassifier(xgbParam).
      setFeaturesCol("features").
      setLabelCol("classIndex")

<console>: 28: error: object spark is not a member of package ml.dmlc.xgboost4j.scala